In [1]:
import os
import cv2
import shutil
import pickle
import pandas as pd
import numpy as np
from tkinter import Tk, Button, messagebox, simpledialog
from datetime import datetime
import time
from deepface import DeepFace

2024-07-16 18:57:35.674190: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# Global variables
model_name = 'Facenet512'
detector_backend = 'mtcnn'
frame_interval = 7 
embeddings_dict = {}

# Load existing embeddings dictionary if available
if os.path.exists('embeddings_dict.pkl'):
    with open('embeddings_dict.pkl', 'rb') as f:
        embeddings_dict = pickle.load(f)

def preprocess_image(image):
    # Apply Gaussian smoothing
    smoothed_image = cv2.GaussianBlur(image, (5, 5), 0)
    return smoothed_image
    
def open_webcam(): # Connnect to webcam
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FPS, 15) # 15 FPS combine with skip every 7 frame -> each second only process about 2 frames
    return cap

def close_webcam(cap): # Disconnnect to webcam
    cap.release()
    cv2.destroyAllWindows()

def check_attendance():
    cap = open_webcam()
    frame_count = 0 # For skip frames -> reduce computational cost

    def recognize(): # Recognize face using MTCNN and Facenet
        nonlocal frame_count
        while True:
            ret, frame = cap.read()
            if not ret:
                messagebox.showerror("Error", "Failed to capture image!")
                close_webcam(cap)
                return

            frame_count += 1
            if frame_count % frame_interval == 0:
                try:
                    preprocessed_frame = preprocess_image(frame)
                    embedding = DeepFace.represent(img_path=frame, model_name=model_name, detector_backend=detector_backend, enforce_detection=False)[0]["embedding"]
                    cosine_min = float('inf')
                    matched_student = None

                    for student_info, stored_embedding in embeddings_dict.items():
                        cosine_similarity = 1 - (np.dot(embedding, stored_embedding) / (np.linalg.norm(embedding) * np.linalg.norm(stored_embedding)))
                        if cosine_similarity < cosine_min:
                            cosine_min = cosine_similarity
                            matched_student = student_info

                    if matched_student and cosine_min < 0.3: 
                        student_name, student_code = matched_student.split('_')
                        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        messagebox.showinfo("Attendance", f"Attendance check successful!\nName: {student_name}\nCode: {student_code}\nTime: {now}")
                        
                        # Append student information to Excel file
                        attendance_data = {
                            "Name": student_name,
                            "Student Code": student_code,
                            "Time Checked": now
                        }
                        attendance_df = pd.DataFrame([attendance_data])
                        excel_file = 'Attendance.xlsx'

                        if os.path.exists(excel_file):
                            existing_df = pd.read_excel(excel_file)
                            attendance_df = pd.concat([existing_df, attendance_df], ignore_index=True)

                        attendance_df.to_excel(excel_file, index=False)
                        
                    else:
                        messagebox.showinfo("Attendance", "Student not found")
                    break  # Exit the loop after processing
                except Exception as e:
                    print(f"Error: {e}")
                    messagebox.showerror("Error", f"An error occurred: {e}")
                    break  # Exit the loop on error

            cv2.imshow('Checking attendance', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    recognize()
    close_webcam(cap)

def add_student():
    cap = open_webcam()
    
    student_name = simpledialog.askstring("Input", "Enter student name to be add:")
    student_code = simpledialog.askstring("Input", "Enter student code to be add:")
    student_info = f"{student_name}_{student_code}"

    if student_info in embeddings_dict:
        messagebox.showerror("Error", "Student already exists!")
        close_webcam(cap)
        return

    messagebox.showinfo("Preparaion Step", "Please face straight to the camera!")
    time.sleep(3)
    messagebox.showinfo("Face Scanning Step", "Starting scanning face")
    time.sleep(1)
    ret, frame = cap.read()
    if ret:
        try:
            embedding = DeepFace.represent(img_path=frame, model_name=model_name, detector_backend=detector_backend, enforce_detection=False)[0]["embedding"]
            embeddings_dict[student_info] = embedding
            with open('embeddings_dict.pkl', 'wb') as f:
                pickle.dump(embeddings_dict, f)
            messagebox.showinfo("Success", "Student added successfully!")
        except Exception as e:
            print(f"Error: {e}")
            messagebox.showerror("Error", f"An error occurred: {e}")
    else:
        messagebox.showerror("Error", "Failed to capture image!")

    close_webcam(cap)

def delete_student():
    student_name = simpledialog.askstring("Input", "Enter student name to delete:")
    student_code = simpledialog.askstring("Input", "Enter student code to delete:")
    student_info = f"{student_name}_{student_code}"

    if student_info in embeddings_dict:
        del embeddings_dict[student_info]
        with open('embeddings_dict.pkl', 'wb') as f:
            pickle.dump(embeddings_dict, f)
        messagebox.showinfo("Success", "Student deleted successfully!")
    else:
        messagebox.showerror("Error", "Student not found!")

# GUI Setup
root = Tk()
root.title("Student Attendance System")
root.geometry("800x500")  # Set the size of the window

btn_check_attendance = Button(root, text="Check Attendance", command=check_attendance, width=20, height=2)
btn_check_attendance.pack(pady=10)

btn_add_student = Button(root, text="Add Student", command=add_student, width=20, height=2)
btn_add_student.pack(pady=10)

btn_delete_student = Button(root, text="Delete Student", command=delete_student, width=20, height=2)
btn_delete_student.pack(pady=10)

root.mainloop()


1/1 [==============================] - 0s 24ms/step


In [2]:
attandance = pd.read_excel('Attendance.xlsx')
attandance

,Name,Student Code,Time Checked
0,Pham Trung Tung,HE187331,2024-07-16 16:28:10
1,hieu,He,2024-07-16 16:29:25
2,hieu,He,2024-07-16 16:30:20
3,Pham Trung Tung,HE187331,2024-07-16 16:30:42
4,Phuc,HHR11,2024-07-16 16:31:25
